In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/clay/userdata/nia-sound-event-1


In [5]:
import json
import torchaudio
from pathlib import Path
from copy import copy
from docx.api import Document
import soundfile as sf
import librosa
from pydub.utils import mediainfo
from tqdm.auto import tqdm
import pandas as pd

from data_utils import NeutuneSet, OnFlyAudio

In [10]:
# dataset = NeutuneSet('/home/teo/userdata/nia_dataset/field_recording')
# dataset = NeutuneSet('/home/teo/label-studio_files/nia_dataset/')
dataset = OnFlyAudio('/home/clay/label_studio_files/data_v4/04/')

In [14]:
from pandas import read_table

from meta_utils import MetaCreator

meta_creator = MetaCreator('vocab.json')
# v1 : 496 63 // 1864 
entire_meta = [meta_creator.create_meta_for_wav(wav) for wav in tqdm(sorted(dataset.wav_list))]

  0%|          | 0/1808 [00:00<?, ?it/s]

Correct middle: 3 / Current middle: 4 / Current Fine: 57
Correct middle: 3 / Current middle: 4 / Current Fine: 57
Correct middle: 3 / Current middle: 4 / Current Fine: 57
Correct middle: 3 / Current middle: 4 / Current Fine: 57
Correct middle: 3 / Current middle: 4 / Current Fine: 57
Correct middle: 3 / Current middle: 4 / Current Fine: 57


In [15]:
error_list = meta_creator.error_list
dict_list = meta_creator.error_dict
print(len(error_list))
dict_list

233


defaultdict(list,
            {"Error occured while handling the corresponding xlsx: expected <class 'str'>": ['fs-2-3-56-1-고수완.wav',
              'fs-2-3-56-1-김규동.wav',
              'fs-2-3-56-1-이상화.wav',
              'fs-2-3-56-2-고수완.wav',
              'fs-2-3-56-2-김규동.wav',
              'fs-2-3-56-2-이상화.wav',
              'fs-2-3-56-3-고수완.wav',
              'fs-2-3-56-3-김규동.wav',
              'fs-2-3-56-3-이상화.wav',
              'fs-2-3-56-4-고수완.wav',
              'fs-2-3-56-4-김규동.wav',
              'fs-2-3-56-4-이상화.wav',
              'fs-2-3-56-5-고수완.wav',
              'fs-2-3-56-5-김규동.wav',
              'fs-2-3-56-5-이상화.wav',
              'fs-2-3-56-6-고수완.wav',
              'fs-2-3-56-6-김규동.wav',
              'fs-2-3-56-6-이상화.wav',
              'fs-2-3-57-1-고수완.wav',
              'fs-2-3-57-1-김규동.wav',
              'fs-2-3-57-1-이상화.wav',
              'fs-2-3-57-2-고수완.wav',
              'fs-2-3-57-2-김규동.wav',
              'fs-2-3-57-2-이상화.wav',
              '

In [20]:
prediction_path = Path('translated_meta_prediction_threshold0.05_0to1573.json')
with open(prediction_path, 'r') as f:
  pred_json = json.load(f)
len(pred_json)

1572

In [21]:
docx_path = '/home/clay/label_studio_files/data_v4/03/metadata/fs-2-3-54-1-가민지.docx'
meta_creator.read_table_from_docx(docx_path)

{'분류 번호': '2-3-54',
 '제목': '서부 총잡이 도시',
 '녹음/제작자 성명': '가민지',
 '녹음/제작 연/월/일': '2022.07.31',
 '사용 기자재': 'Ableton Live DAW',
 '오디오 포맷': '스테레오 - wav (96k, 24bit)',
 '제작 장소': '서산시 동문동',
 '보안 이슈': '없음',
 '자료 제공자 정보': '성명: 000\n연령: 000'}

In [9]:
document = Document(docx_path)
table = document.tables[0]
table_dict = {row.cells[0].text:row.cells[1].text for row in table.rows[1:]}

table_dict

{'분류 번호': '2-3-54',
 '제목': '서부 총잡이 도시',
 '녹음/제작자 성명': '가민지',
 '녹음/제작 연/월/일': '2022.07.31',
 '사용 기자재': 'Ableton Live DAW',
 '오디오 포맷': '스테레오 - wav (96k, 24bit)',
 '제작 장소': '서산시 동문동',
 '보안 이슈': '없음',
 '자료 제공자 정보': '성명: 000\n연령: 000'}

In [10]:
for row in table.rows:
  print(row.cells[1].text)


2-3-54
서부 총잡이 도시
가민지
2022.07.31
Ableton Live DAW
스테레오 - wav (96k, 24bit)
서산시 동문동
없음
성명: 000
연령: 000


In [110]:
count = 0

document = Document(docx_path)
table = document.tables[0]
for row in table.rows:
  print(row)
  try:
    print(row.cells[2].text)
  except:
    continue
  
  '''
  if count != 0:
    print(row.cells)
  count += 1
  '''


분류 번호
마곡 코트야드 호텔 뷔페 옆 큰 공간
녹음 연/월/일
Zoom H6 레코더, 윈드쉴드, 헤드셋
오디오 길이
서울시 강서구 마곡중앙 12로
음원으로부터의 거리
약간 흐림
보안 이슈
호텔 음악소리 살짝 섞임


In [18]:
excel_path = '/home/clay/label_studio_files/data_v4/04/metadata/fs-2-3-56-1-이상화.xlsx'
detailed_meta = pd.read_excel(excel_path, index_col=0)
detailed_meta

TypeError: expected <class 'str'>

In [17]:
excel_path = '/home/clay/userdata/nia_dataset/data_v4/01/metadata/fs-2-3-56-1-김규동.xlsx'
detailed_meta = pd.read_excel(excel_path, index_col=0)
detailed_meta

FileNotFoundError: [Errno 2] No such file or directory: '/home/clay/userdata/nia_dataset/data_v4/01/metadata/fs-2-3-56-1-김규동.xlsx'

In [115]:
excel_path = '/home/clay/userdata/nia_dataset/data_v4/01/metadata/fs-2-4-75-1-김수환.xlsx'
detailed_meta = pd.read_excel(excel_path, index_col=0)
detailed_meta

,Unnamed: 1
분류 번호,1(공간/환경)-1(항공기)-002(제트기)
제목,제트 엔진 소리
녹음/제작자 성명,김수환
녹음/제작 연/월/일,2022.07.17.
사용 기자재,Ableton Live DAW
오디오 포맷,"스테레오 - wav (96k, 24bit) / mp3"
오디오 길이,105초
제작 장소,서울시 금호로 107
보안 이슈,없음
자료 제공자 정보,Soundly


In [17]:
event_id = 0
dataset = OnFlyAudio('/home/clay/userdata/nia_dataset/data_v4/01/')
dataset.path, dataset.mp3_list[0]

(PosixPath('/home/clay/userdata/nia_dataset/data_v4/01'),
 PosixPath('/home/clay/userdata/nia_dataset/data_v4/01/mp3/fs-2-4-75-1-가민지.mp3'))

In [19]:
event_id = 0
sample_path = dataset.mp3_list[event_id].relative_to(dataset.path)
print(sample_path)
sample_path = '01' / sample_path
sample_path

mp3/fs-2-4-75-1-가민지.mp3


PosixPath('01/mp3/fs-2-4-75-1-가민지.mp3')

In [ ]:
import numpy as np

class AudioFeatureExtractor:
  def __init__(self):
    pass 

  def get_audio_features(self, y:np.ndarray, sr:int):
    spec = self.get_spec(y, sr)
    
    spec_centroid = self.get_mean_and_std_of_audio_feature(spec, sr, librosa.feature.spectral_centroid, 'spectral centroid')
    
    spec_centroid.update(self.get_mean_and_std_of_audio_feature(spec, sr, librosa.feature.spectral_bandwidth, 'spectral bandwidth'))
    spec_centroid.update(self.get_mean_and_std_of_audio_feature(spec, sr, librosa.feature.spectral_contrast, 'spectral contrast'))
    spec_centroid.update(self.get_mean_and_std_of_audio_feature(spec, sr, librosa.feature.spectral_rolloff, 'spectral roll off'))
    spec_centroid.update(self.get_mean_and_std_of_audio_feature_wo_sr(y, librosa.feature.spectral_flatness, 'spectral flatness'))
    spec_centroid.update(self.get_mean_and_std_of_audio_feature_wo_sr(y, librosa.feature.zero_crossing_rate, 'zero crossing rate'))
    spec_centroid.update(self.get_mean_and_std_of_audio_feature_wo_sr(y, librosa.feature.rms, 'rms'))

    return spec_centroid
  
  def get_spec(self, y, sr):
    return np.abs(librosa.stft(y))
  
  def get_mean_and_std_of_audio_feature(self, y, sr, audio_feature, audio_feature_name:str):
    audio_features = audio_feature(S = y, sr = sr)
    return {f'{audio_feature_name} mean': np.mean(audio_features) , f'{audio_feature_name} std':np.std(audio_features) }
  
  def get_mean_and_std_of_audio_feature_wo_sr(self, y, audio_feature, audio_feature_name:str):
    audio_features = audio_feature(y = y)
    return {f'{audio_feature_name} mean': np.mean(audio_features) , f'{audio_feature_name} std':np.std(audio_features) }

  def read_soundfile(self, soundfile:sf.SoundFile):
    y = soundfile.read()
    y = y.mean(1)
    return y, soundfile.samplerate

  def __call__(self, soundfile:sf.SoundFile):
    y, sr = self.read_soundfile(soundfile)
    return self.get_audio_features(y, sr)

extractor = AudioFeatureExtractor()
ob = sf.SoundFile(dataset.wav_list[290])

# extractor(ob)

In [ ]:
example_meta = entire_meta[290]
example_meta['오디오 음향 특성'] = extractor(ob)

In [ ]:
with open("example_meta.json", 'w') as f:
  json.dump(example_meta, f, ensure_ascii=False)
# example_meta

In [ ]:
import pandas as pd
pd.DataFrame([(x['소분류'], x['녹음 방식'], x['파일 이름'], x['길이']) for x in entire_meta]).to_csv('length.csv')

In [ ]:
len(dataset.wav_list)

In [ ]:
[error['file'] for error in meta_creator.error_list if error['error'] == 'Error occured while handling the corresponding docx']

In [ ]:
meta_creator.error_list

In [ ]:
entire_meta[290]

In [ ]:
for meta in entire_meta:
  if '필드 레코딩 메타' in meta:
    print(meta['필드 레코딩 메타']['소리의 강도'])

In [ ]:
test = Document("/home/teo/userdata/nia_dataset/field_recording/5팀 이상화 1주차/metadata/1-2-5-1-김민규.docx")
table = test.tables[0]

for row in table.rows:
  print(row.cells[0].text, '//', row.cells[1].text)

In [ ]:
meta_creator.read_table_from_docx("/home/teo/userdata/nia_dataset/field_recording/4팀 김양우 1주차/metadata/1-1-3-1-한은광.docx")

In [ ]:
Path("/home/teo/userdata/nia_dataset/field_recording/5팀 이상화 1주차/metadata/1-2-7-6-백승렬.docx").exists()

In [ ]:
table = meta_creator.read_table_from_docx('1-1-1-1-송영재.docx')
table

In [ ]:
import pandas as pd
pd.DataFrame(table, index=['']).T.to_excel('1-1-1-1-송영재.xlsx')

In [ ]:
pd.read_excel('/home/teo/label-studio_files/nia_dataset/__DATA_v4/01/metadata/r-1-1-4-11-백승렬.xlsx')

In [ ]:
!pip install openpyxl

In [ ]:
meta_creator.vocab['idx2title']

In [ ]:
mediainfo("/home/teo/userdata/nia_dataset/field_recording/5팀 이상화 1주차/mp3/1-2-7-6-백승렬.mp3")

In [ ]:
import docx

def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

test = getText('1-1-1-1-송영재.docx')

In [ ]:
from docx.api import Document

# Load the first table from your document. In your example file,
# there is only one table, so I just grab the first one.
document = Document('1-1-1-1-송영재.docx')
table = document.tables[0]

In [ ]:
data = []

keys = None
for i, row in enumerate(table.rows):
    text = (cell.text for cell in row.cells)

    # Establish the mapping based on the first row
    # headers; these will become the keys of our dictionary
    if i == 0:
        keys = tuple(text)
        continue

    # Construct a dictionary for this row, mapping
    # keys to values for this row
    row_data = dict(zip(keys, text))
    data.append(row_data)
data[0]['']